In [33]:
import certifi
import ssl
import geopy.geocoders
from geopy.geocoders import GoogleV3

ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx

In [34]:
f = open("gmaps_key.txt", "r") 
key = f.read()

In [35]:
geolocator = GoogleV3(api_key=key, user_agent="onhb-geo-scrapper", timeout=3)

In [36]:
location = geolocator.geocode("Campinas, SP, Brazil")
print((location.latitude, location.longitude))

(-22.9329252, -47.073845)


In [37]:
import json

with open('data.json') as json_file:  
    data = json.load(json_file)

In [32]:
i = 1
for city in data['data']:
  if city['lat'] == '' and i < 10:
    s = city['cidade'] + ', ' + city['uf'] + ', Brazil'
    try:
      location = geolocator.geocode(s)
    except Exception as e:
      print(f'#{i} -> {s} -> ERROR: {}')
    print(f'#{i} -> {s} -> {location.latitude} / {location.longitude}')
    city['lat'] = location.latitude
    city['long'] = location.longitude
    i += 1

GeocoderTimedOut: Service timed out